In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.singleton_classifier import SingletonClassifierModelBuilder
from functools import reduce

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
data = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

data.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,num_words,previous_words,next_words,is_singleton
0,1,[1],0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,1,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",0,0,2,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 10, 12, 13]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]",1,0,3,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 10, 12, 13, 14, 15, 16]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",0,0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 10, 12, 13, 14, 15, 16, 17, 18]","[0.0, 1.0]"
4,5,"[10, 12]",0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",0,0,2,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[13, 14, 15, 16, 17, 18, 4, 5, 19, 20]","[0.0, 1.0]"


In [4]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person', 'num_words']]

data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
label = np.vstack(data.is_singleton)

# Build Model

## Words

In [20]:
words_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [15]:
words_model = words_model_builder.create_model()

In [16]:
words_model.fit([data_text], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 4s 707us/step - loss: 0.3135 - acc: 0.8440
Epoch 2/10
5032/5032 [==============================] - 2s 418us/step - loss: 0.2049 - acc: 0.8971
Epoch 3/10
5032/5032 [==============================] - 2s 444us/step - loss: 0.1746 - acc: 0.9161
Epoch 4/10
5032/5032 [==============================] - 2s 435us/step - loss: 0.1620 - acc: 0.9279
Epoch 5/10
5032/5032 [==============================] - 2s 430us/step - loss: 0.1421 - acc: 0.9364
Epoch 6/10
5032/5032 [==============================] - 2s 443us/step - loss: 0.1346 - acc: 0.9364
Epoch 7/10
5032/5032 [==============================] - 2s 456us/step - loss: 0.1197 - acc: 0.9390
Epoch 8/10
5032/5032 [==============================] - 2s 460us/step - loss: 0.1149 - acc: 0.9384
Epoch 9/10
5032/5032 [==============================] - 2s 433us/step - loss: 0.1125 - acc: 0.9402
Epoch 10/10
5032/5032 [==============================] - 2s 447us/step - loss: 0.1074 - acc: 0.9416


In [17]:
#words_model.save('models/singleton_classifiers/words.model')

## Context

In [4]:
context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [5]:
context_model = context_model_builder.create_model()

In [28]:
context_model.fit([data_previous_words, data_next_words], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 7s 1ms/step - loss: 0.4773 - acc: 0.8239
Epoch 2/10
5032/5032 [==============================] - 5s 915us/step - loss: 0.4091 - acc: 0.8400
Epoch 3/10
5032/5032 [==============================] - 4s 840us/step - loss: 0.3512 - acc: 0.8466
Epoch 4/10
5032/5032 [==============================] - 4s 792us/step - loss: 0.3016 - acc: 0.8674
Epoch 5/10
5032/5032 [==============================] - 4s 778us/step - loss: 0.2544 - acc: 0.8857
Epoch 6/10
5032/5032 [==============================] - 4s 776us/step - loss: 0.2226 - acc: 0.9066
Epoch 7/10
5032/5032 [==============================] - 4s 827us/step - loss: 0.1793 - acc: 0.9199
Epoch 8/10
5032/5032 [==============================] - 4s 819us/step - loss: 0.1689 - acc: 0.9326
Epoch 9/10
5032/5032 [==============================] - 4s 882us/step - loss: 0.1294 - acc: 0.9473
Epoch 10/10
5032/5032 [==============================] - 4s 890us/step - loss: 0.1080 - acc: 0.9563


In [9]:
#context_model.save('models/singleton_classifiers/context.model')

## Syntactic

In [11]:
syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    syntactic_features_num=data_syntactic.shape[1]
)

In [12]:
syntactic_model = syntactic_model_builder.create_model()

In [13]:
syntactic_model.fit([data_syntactic], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 2s 316us/step - loss: 0.3927 - acc: 0.8388
Epoch 2/10
5032/5032 [==============================] - 0s 97us/step - loss: 0.2861 - acc: 0.8388
Epoch 3/10
5032/5032 [==============================] - 0s 99us/step - loss: 0.2647 - acc: 0.8402
Epoch 4/10
5032/5032 [==============================] - 0s 97us/step - loss: 0.2599 - acc: 0.8557
Epoch 5/10
5032/5032 [==============================] - 0s 98us/step - loss: 0.2556 - acc: 0.8543
Epoch 6/10
5032/5032 [==============================] - 0s 98us/step - loss: 0.2538 - acc: 0.8579
Epoch 7/10
5032/5032 [==============================] - 0s 98us/step - loss: 0.2503 - acc: 0.8613
Epoch 8/10
5032/5032 [==============================] - 0s 98us/step - loss: 0.2542 - acc: 0.8569
Epoch 9/10
5032/5032 [==============================] - 0s 98us/step - loss: 0.2495 - acc: 0.8661
Epoch 10/10
5032/5032 [==============================] - 1s 117us/step - loss: 0.2468 - acc: 0.8635


In [15]:
#syntactic_model.save('models/singleton_classifiers/syntactic.model')

## Words + Context

In [16]:
words_context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [17]:
words_context_model = words_context_model_builder.create_model()

In [19]:
words_context_model.fit([data_text, data_previous_words, data_next_words], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 8s 2ms/step - loss: 0.3226 - acc: 0.8543
Epoch 2/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1832 - acc: 0.9179
Epoch 3/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1446 - acc: 0.9388
Epoch 4/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1312 - acc: 0.9459
Epoch 5/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0944 - acc: 0.9634
Epoch 6/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0738 - acc: 0.9712
Epoch 7/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0576 - acc: 0.9762
Epoch 8/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0540 - acc: 0.9807
Epoch 9/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0404 - acc: 0.9877
Epoch 10/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0445 - acc: 0.9859


In [21]:
#words_context_model.save('models/singleton_classifiers/words_context.model')

## Words + Syntactic

In [22]:
words_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [23]:
words_syntactic_model = words_syntactic_model_builder.create_model()

In [24]:
words_syntactic_model.fit([data_text, data_syntactic], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 4s 874us/step - loss: 0.3004 - acc: 0.8531
Epoch 2/10
5032/5032 [==============================] - 3s 511us/step - loss: 0.2120 - acc: 0.8923
Epoch 3/10
5032/5032 [==============================] - 3s 542us/step - loss: 0.1677 - acc: 0.9179
Epoch 4/10
5032/5032 [==============================] - 3s 507us/step - loss: 0.1510 - acc: 0.9289
Epoch 5/10
5032/5032 [==============================] - 3s 502us/step - loss: 0.1334 - acc: 0.9382
Epoch 6/10
5032/5032 [==============================] - 3s 500us/step - loss: 0.1312 - acc: 0.9372
Epoch 7/10
5032/5032 [==============================] - 2s 497us/step - loss: 0.1195 - acc: 0.9440
Epoch 8/10
5032/5032 [==============================] - 3s 515us/step - loss: 0.1291 - acc: 0.9416
Epoch 9/10
5032/5032 [==============================] - 3s 607us/step - loss: 0.1171 - acc: 0.9438
Epoch 10/10
5032/5032 [==============================] - 3s 532us/step - loss: 0.1129 - acc: 0.9442


In [26]:
#words_syntactic_model.save('models/singleton_classifiers/words_syntactic.model')

## Context + Syntactic

In [27]:
context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [28]:
context_syntactic_model = context_syntactic_model_builder.create_model()

In [29]:
context_syntactic_model.fit([data_previous_words, data_next_words, data_syntactic], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 8s 2ms/step - loss: 0.4713 - acc: 0.8309
Epoch 2/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.3091 - acc: 0.8555
Epoch 3/10
5032/5032 [==============================] - 4s 871us/step - loss: 0.2186 - acc: 0.8965
Epoch 4/10
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1714 - acc: 0.9261
Epoch 5/10
5032/5032 [==============================] - 5s 936us/step - loss: 0.1548 - acc: 0.9352
Epoch 6/10
5032/5032 [==============================] - 5s 935us/step - loss: 0.1167 - acc: 0.9533
Epoch 7/10
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1010 - acc: 0.9622
Epoch 8/10
5032/5032 [==============================] - 5s 948us/step - loss: 0.0735 - acc: 0.9736
Epoch 9/10
5032/5032 [==============================] - 5s 932us/step - loss: 0.0665 - acc: 0.9771
Epoch 10/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0577 - acc: 0.9811


In [31]:
#context_syntactic_model.save('models/singleton_classifiers/context_syntactic.model')

## Words + Context + Syntactic

In [32]:
words_context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [33]:
words_context_syntactic_model = words_context_syntactic_model_builder.create_model()

In [34]:
words_context_syntactic_model.fit([data_text, data_previous_words, data_next_words, data_syntactic], label, epochs=10)

Epoch 1/10
5032/5032 [==============================] - 10s 2ms/step - loss: 0.4057 - acc: 0.8146
Epoch 2/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.2110 - acc: 0.9014
Epoch 3/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1511 - acc: 0.9384
Epoch 4/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1274 - acc: 0.9499
Epoch 5/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1006 - acc: 0.9585
Epoch 6/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0842 - acc: 0.9714
Epoch 7/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0789 - acc: 0.9738
Epoch 8/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0575 - acc: 0.9807
Epoch 9/10
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0538 - acc: 0.9827
Epoch 10/10
5032/5032 [==============================] - 7s 1ms/step - loss: 0.0582 - acc: 0.9839


In [36]:
#words_context_syntactic_model.save('models/singleton_classifiers/words_context_syntactic.model')